In [ ]:
from QHT import QHT
from exactqcnn import ExactQCNN
from matplotlib import pyplot as plt
import numpy as np
import json

In [ ]:
exactqcnn = ExactQCNN(15)
ins = QHT(None, exactqcnn, None)

In [ ]:
ins.calc_error_separable_CSqNeyman(n=1, seed=2, 
                                   a_li=[i/10 for i in range(-10,11)],shots=20*60*2, 
                                   n_ent=3, data="test2", haar="haar",
                                   read_json=False,read_npz=True)

## Create random unitaries

In [ ]:
def save_random(mode, qubits):
    import os
    import re
    from qiskit.quantum_info import random_clifford, random_unitary
    from tqdm.notebook import tqdm
    print(f"mode={mode}, qubits={qubits}")
    
    if mode=="clifford":
        # フォルダ内のファイルを取得
        folder = "./random_unitary/random_clifford/"
        files = os.listdir(folder)
        max_i = 0  # 最大の i の初期値を設定
        # ファイル名から i の値を抽出して比較
        for file_name in files:
            match = re.match(fr"{qubits}qubits_40x2x10000num\((\d+)\).npy", file_name)
            if match:
                i = int(match.group(1))
                max_i = max(max_i, i)
        fname = f"{qubits}qubits_40x2x10000num({max_i+1}).npy"
        U_li = np.array([random_clifford(qubits).to_matrix() for _ in tqdm(range(40*2*10000))])
        np.save(folder+fname, U_li)
        # fname = f"{qubits}qubits_40x2x10num({max_i+1}).npz"
        # U_li = [random_clifford(qubits).to_matrix() for _ in tqdm(range(40*2*10))]
        # np.savez_compressed(folder+fname, *U_li)

    elif mode=="haar":
        # フォルダ内のファイルを取得
        folder = "./random_unitary/random_haar/"
        files = os.listdir(folder)
        max_i = 0  # 最大の i の初期値を設定
        # ファイル名から i の値を抽出して比較
        for file_name in files:
            match = re.match(fr"{qubits}qubits_40x2x10000num\((\d+)\).npy", file_name)
            if match:
                i = int(match.group(1))
                max_i = max(max_i, i)
        fname = f"{qubits}qubits_40x2x10000num({max_i+1}).npy"
        U_li = np.array([random_unitary(2**qubits).data for _ in tqdm(range(40*2*10000))])
        np.save(folder+fname, U_li)
for _ in range(5):
    save_random("haar", 1)

## Plot

In [ ]:
class PlotData:
    def __init__(self, qht:QHT):
        self.qht = qht
        self.results_list_alpha_vs_beta = []
        self.results_list_n_vs_logbeta = []
        self.results_list_beta_vs_n = []
        self.results_list_shots_vs_n = [] 
    def get_results_alpha_vs_beta(self, n, data="test1"):
        results_error_li = []
        results_error_point = []
        settings_dict = []
        results_error_li.append(self.qht.calc_error_separable_OP(n, alpha_min_li=[i/100 for i in range(1,100)],
                                                                 read_json=True, data=data))
        settings_dict.append({"color":"b","linestyle":"-","label":"cNeyman(OP)"})
        for x in [8, 15, 30, 60, 120, 240]:
            results_error_li.append(self.qht.calc_error_separable_CSqNeyman(n, seed=2, 
                                    a_li=[i/10 for i in range(-10,11)],shots=20*x*2, 
                                    n_ent=3, data=data, haar="haar",
                                    read_json=True,read_npz=False))
            settings_dict.append({"color":"brown","marker":".","linestyle":"--","label":"tomography"})
        # results_error_li.append(self.qht.calc_error_separable_PartialCSqNeyman(n, 
        #                         a_li=[i/10 for i in range(-10,11)],
        #                         n_ent=6, data=data,
        #                         read_json=False,read_npz=False))
        # settings_dict.append({"color":"brown","marker":".","linestyle":"-","label":"PartialCS"})
        results_error_li.append(self.qht.calc_error_separable_qcnn(n, shots_per_rho=None,
                                                                   alpha_min_li=[i/100 for i in range(1,100)],
                                                                   index=150, read_json=True, data=data,
                                                                   qcnn_mode="qcnn1"))
        settings_dict.append({"color":"m","linestyle":"-","label":"qcnn"})
        results_error_li.append(self.qht.calc_error_separable_exactqcnn(n, alpha_min_li=[i/100 for i in range(1,100)], read_json=True, data=data))
        settings_dict.append({"color":"g","linestyle":"-","label":"exactqcnn"})
        self.results_list_alpha_vs_beta.append([n, results_error_li, results_error_point, settings_dict])
    def get_results_beta_vs_n(self, n_max, data="test1"):
        n_li = list(range(1,n_max+1))
        results_error_li = []
        results_error_point = []
        settings_dict = []
        results_error_li.append(self.qht.calc_error_separable_OP(n_li, alpha_min_li=[i/500 for i in range(1,500)],
                                                                 read_json=True, data=data))
        settings_dict.append({"color":"b", "linestyle":"", "marker":".", "label":"cNeyman(OP)"})
        results_error_li.append(self.qht.calc_error_separable_CSqNeyman(n_li, seed=2, 
                                a_li=[i/10 for i in range(-10,11)],shots=20*60*2,
                                n_ent=2, data=data, haar="haar",
                                read_json=True,read_npz=True))
        settings_dict.append({"color":"brown", "linestyle":"", "marker":".", "label":"tomography"})
        results_error_li.append(self.qht.calc_error_separable_PartialCSqNeyman(n_li, 
                                a_li=[i/10 for i in range(-10,11)],
                                n_ent=2, data=data,
                                read_json=True,read_npz=True))
        settings_dict.append({"color":"brown","linestyle":"","marker":"x","label":"PartialCS"})
        results_error_li.append(self.qht.calc_error_separable_qcnn(n_li, shots_per_rho=None, 
                                                                   alpha_min_li=[i/500 for i in range(1,500)],
                                                                   index=150, read_json=True, data=data,
                                                                   qcnn_mode="qcnn1"))
        settings_dict.append({"color":"m", "linestyle":"", "marker":".", "label":"qcnn"})
        results_error_li.append(self.qht.calc_error_separable_exactqcnn(n_li, alpha_min_li=[i/500 for i in range(1,500)], read_json=True, data=data))
        settings_dict.append({"color":"g", "linestyle":"", "marker":".", "label":"exactqcnn"})
        self.results_list_beta_vs_n.append([results_error_li, results_error_point, settings_dict])
    def get_results_shots_vs_n(self, n_max, data="test1"):
        n_li = list(range(1,n_max+1))
        shots_li = []
        results_error_li = []
        results_error_point = []
        settings_dict = []
        shots_li.append([320,600,1200,2400])
        results_error_li.append([self.qht.calc_error_separable_CSqNeyman(n_li, seed=2, 
                                a_li=[i/10 for i in range(-10,11)], shots=shots,
                                n_ent=2, data=data,
                                read_json=True,read_npz=True) for shots in shots_li[-1]])
        settings_dict.append({"color":"brown", "linestyle":"--", "marker":"o", "label":"tomography"})
        shots_per_rho = 500
        shots_li.append([i*20*shots_per_rho*2 for i in range(0,151,10)])
        results_error_point.append([self.qht.calc_error_separable_qcnn(n_li, shots_per_rho=shots_per_rho, 
                                                                      index=shots//(20*shots_per_rho*2), read_json=True, data=data,
                                                                      qcnn_mode="qcnn2") for shots in shots_li[-1]])
        settings_dict.append({"color":"m", "linestyle":"--", "marker":"o", "label":"qcnn"})
        self.results_list_shots_vs_n.append([shots_li, results_error_li, results_error_point, settings_dict])

ins_plotdata = PlotData(qht=ins)

##### $\beta_n$ vs. $\alpha_n$

In [15]:
ins_plotdata.get_results_alpha_vs_beta(1, "test2")

In [ ]:
ins.plot_alpha_vs_beta(*ins_plotdata.results_list_alpha_vs_beta[0])

##### $n$ vs. $\beta_n$

In [17]:
ins_plotdata.get_results_beta_vs_n(50, "test1")

In [ ]:
for alpha in [0.3,0.1,0.05]:
    ins.plot_beta_vs_n(alpha, *ins_plotdata.results_list_beta_vs_n[0])

##### $n$ vs. training shots

In [79]:
ins_plotdata.get_results_shots_vs_n(500, "test1")

In [ ]:
for a in [0.3,0.1,0.05]:
    ins.plot_shots_vs_n(a, a, *ins_plotdata.results_list_shots_vs_n[0])

##### testloss vs. training shots

In [ ]:
for x in [8,15,30,60,120,240]:
    ins.calc_probnt_separable_CSqNeyman(seed=2, a_=0, shots=20*x*2,
                                        n_ent=3, data="test2", haar="clifford",
                                        read_json=False)

In [ ]:
def plot_shots_vs_testloss(qht:QHT, data="test1", order=[], scale="linear"):
    if data=="test1":
        label_i = qht.label_i_test1
    elif data=="test2":
        label_i = qht.label_i_test2
    label_i[label_i!=0] = 1
    shots_li = []
    testloss_li = []
    settings_dict = []

    shots_li.append([320,600,1200,2400,4800,9600])
    probnt_li = [qht.calc_probnt_separable_CSqNeyman(seed=2, a_=0, shots=shots, 
                                                     n_ent=2, data=data, haar="haar",
                                                     read_json=True) for shots in shots_li[-1]]
    testloss_li.append([np.sum((probnt-label_i)**2)/len(probnt) for probnt in probnt_li])
    settings_dict.append({"color":"brown","marker":"o","linestyle":"--","label":"tomography"})
    
    # shots_per_rho = 1000
    # qcnn_mode = "qcnn1"
    # shots_li.append([i*20*shots_per_rho*2 for i in range(0,151,10)])
    # probnt_li = []
    # for shots in shots_li[-1]:
    #     with open(f"./json_data/{qcnn_mode}(L={qht.n_qubits}_shots={shots_per_rho},{data}).json") as f:
    #         json_dict = json.load(f)
    #     ave_qcnn_json = json_dict["expectation_list"][f"index={shots//(20*shots_per_rho*2)}"]
    #     probnt_li.append(ave_qcnn_json)
    # testloss_li.append([np.sum((probnt-label_i)**2)/len(probnt) for probnt in probnt_li])
    # settings_dict.append({"color":"brown", "linestyle":"--", "marker":".", "label":"qcnn"})

    qcnn_mode = "lowesaqcnn1"
    shots_li.append([20*i for i in (100, 250, 500)])
    probnt_li = []
    for shots in shots_li[-1]:
        with open(f"./json_data/{qcnn_mode}(L={qht.n_qubits}_shots={shots//20},{data}).json") as f:
            json_dict = json.load(f)
        ave_qcnn_json = json_dict["expectation_list"][f"index={300}"]
        probnt_li.append(ave_qcnn_json)
    testloss_li.append([np.sum((probnt-label_i)**2)/len(probnt) for probnt in probnt_li])
    settings_dict.append({"color":"m", "linestyle":"--", "marker":"o", "label":"low-wegiht qcnn"})

    fig = plt.figure(figsize=(4.8,4.8))
    # 設定
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams["font.size"] = 20
    plt.rcParams['xtick.direction'] = 'in'
    plt.rcParams['ytick.direction'] = 'in'
    for i, testloss in enumerate(testloss_li):
        plt.plot(shots_li[i],testloss, **settings_dict[i])

    plt.xscale(scale)
    plt.xlabel("training shots")
    plt.ylabel("validation loss (MSE)")
    plt.ticklabel_format(style="sci", axis="x", scilimits=(3,3))
    # plt.title("Validation Loss vs. Training Shots")
    # ordering legend
    # if order:
    #     handles, labels = plt.gca().get_legend_handles_labels()
    #     plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])
    # else:
    #     plt.legend()

    plt.show(fig)
    plt.close(fig)
plot_shots_vs_testloss(ins, "test1", order=[0])